In [1]:
# Simple analysis to identify Spike mutations associated with GSIAD-annotated Pangolin linages
# We align each
# This analysis requires that you download the "spikeprot" spike protein sequences from GISAID,
# as well as the metadata file from GISAID.
# We cannot store this data in this repo due to licensing.
#
import os
import sys
import re

import tqdm

import pandas
import seaborn
from matplotlib import pyplot

%config InlineBackend.figure_format='retina'

# Yabul is a little library for fasta reading and alignment.
# !pip install yabul
import yabul

In [2]:
spike_df = yabul.read_fasta("spikeprot0223.fasta").rename(columns={"description": "sequence_id"})

# Parse out the EPI ISL ID
spike_df["gisaid_epi_isl"] = spike_df.sequence_id.map(lambda s: re.search("\\|(EPI_ISL_[0-9]+)\\|", s).groups()[0])

metadata_df = pandas.read_csv("metadata_2021-02-24_10-05.tsv.gz", sep=None, engine='python')
spike_df = spike_df.merge(metadata_df, on="gisaid_epi_isl").sort_values("date").drop_duplicates("gisaid_epi_isl").set_index("gisaid_epi_isl")
spike_df = spike_df.rename(columns={"sequence": "unaligned_sequence"})
spike_df

reference, = spike_df.loc[
    spike_df.sequence_id.str.lower().str.contains("wiv04")
].drop_duplicates().index
reference

len(spike_df), reference

(514759, 'EPI_ISL_402124')

In [3]:
reference_seq = spike_df.loc[reference, "unaligned_sequence"]
reference_seq

'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITG

In [4]:
other_seq = spike_df.iloc[-1].unaligned_sequence
other_seq, other_seq == reference_seq, len(reference_seq), len(other_seq)

('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAISGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTYGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIDDTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSHRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPINFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILARLDKVEAEVQIDRLITGRL

In [5]:
pairwise_alignments_df = []
for seq in tqdm.tqdm(spike_df.unaligned_sequence.unique()):
    result = yabul.align_pair(seq, reference_seq)
    pairwise_alignments_df.append((
        seq,
        result.reference,
        result.query
    ))
pairwise_alignments_df = pandas.DataFrame(
    pairwise_alignments_df, columns=["unaligned_sequence", "aligned_ref", "aligned_query"])
pairwise_alignments_df

100%|██████████| 74964/74964 [09:56<00:00, 125.70it/s]


,unaligned_sequence,aligned_ref,aligned_query
0,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSSTRGVYYPDKVFRSS...
1,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...
2,MFVFLFVLPLVSSQCVNLTTRTGIQPGYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVNLTTRTGIQPGYTNSSTRGVYYPDKVFRSS...
3,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...
4,MFVFLFVLPLVSSQCVXXXTRTGIPPGYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVXXXTRTGIPPGYTNSSTRGVYYPDKVFRSS...
...,...,...,...
74959,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
74960,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
74961,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
74962,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [6]:
# Positions are 1-based inclusive

def mut_tuple_to_human_mut(tpl):
    # Helper function for call_muts.
    # Given a tuple describing a mut, return
    # a string giving the human readable version.
    # The input tpl should have one of the following forms:
    # - ("sub", position, reference aa, variant aa)
    # - ("del", start position, end position)
    # - ("ins", position, inserted sequence)
    #
    kind = tpl[0]
    if kind == "sub":
        (pos, ref_aa, aa) = tpl[1:]
        return "p.%s%d%s" % (ref_aa, pos, aa)
    elif kind == "del":
        (start, end) = tpl[1:]
        if start == end:
            return "del.%d" % start
        else:
            return "del.%d-%d" % (start, end)
    elif kind == "ins":
        (pos, seq) = tpl[1:]
        return "ins.%d%s" % (pos, seq)

MUTS_CACHE = {}
def call_muts(reference_seq, seq, exclude_x=True):
    # Given a full length reference seq and a variant seq, both aligned (i.e. same length),
    # return a list of strings giving the mutations in human-readable form.
    
    try:
        return MUTS_CACHE[(reference_seq, seq)]
    except KeyError:
        pass
    
    pos = 1
    assert len(seq) == len(reference_seq)
    results = []
    for (aligned_pos, (aa, ref_aa)) in enumerate(zip(seq, reference_seq)):
        if aa != ref_aa:
            if aa == "X" and exclude_x:
                pass
            elif aa == "-":
                # Deletion
                if results and results[-1][0] == "del" and results[-1][2] == pos - 1:
                    # Merge with deletion of previous
                    results[-1] = ("del", results[-1][1], pos)
                else:
                    # Start a new deletion
                    results.append(("del", pos, pos))
            elif ref_aa == "-":
                # Insertion
                if results and results[-1][0] == "ins" and results[-1][1] == pos:
                    # Merge with previous
                    results.append(("ins", pos, results[-1][2] + aa))
            else:
                # Simple mutation
                results.append(("sub", pos, ref_aa, aa))
        
        if ref_aa != "-":
            pos += 1
                
    human_results = [mut_tuple_to_human_mut(tpl) for tpl in results]
    MUTS_CACHE[(reference_seq, seq)] = human_results
    return human_results
            
pairwise_alignments_df["muts"] = [
    call_muts(row.aligned_ref, row.aligned_query)
    for _, row in tqdm.tqdm(pairwise_alignments_df.iterrows())
]
pairwise_alignments_df["num_muts"] = pairwise_alignments_df["muts"].str.len()
pairwise_alignments_df

74964it [00:23, 3161.68it/s]


,unaligned_sequence,aligned_ref,aligned_query,muts,num_muts
0,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSSTRGVYYPDKVFRSS...,"[p.F32S, p.S50L, p.T76I, p.Q218P, p.E324D, p.R...",30
1,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,"[p.V6F, p.L7V, p.Q23G, p.L24I, p.A27G, p.F32S,...",94
2,MFVFLFVLPLVSSQCVNLTTRTGIQPGYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVNLTTRTGIQPGYTNSSTRGVYYPDKVFRSS...,"[p.V6F, p.L7V, p.Q23G, p.L24I, p.P25Q, p.A27G,...",94
3,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,"[p.V6F, p.L7V, p.Q23G, p.L24I, p.A27G, p.F32S,...",93
4,MFVFLFVLPLVSSQCVXXXTRTGIPPGYTNSSTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVXXXTRTGIPPGYTNSSTRGVYYPDKVFRSS...,"[p.V6F, p.L7V, p.Q23G, p.L24I, p.A27G, p.F32S,...",45
...,...,...,...,...,...
74959,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.D614G, p.A846V]",2
74960,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.A520S, p.D614G]",2
74961,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.S255A, p.A263S, p.A264G, p.Y265W, p.Y266T, ...",8
74962,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[del.69-70, del.144, p.N501Y, p.A570D, p.D614G...",9


In [7]:
annotated_df = spike_df.merge(pairwise_alignments_df, on="unaligned_sequence")
annotated_df

,sequence_id,unaligned_sequence,strain,virus,genbank_accession,date,region,country,division,location,...,authors,url,title,paper_url,date_submitted,purpose_of_sequencing,aligned_ref,aligned_query,muts,num_muts
0,Spike|hCoV-19/bat/Yunnan/RaTG13/2013|2013-07-2...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSSTRGVYYPDKVFRSS...,bat/Yunnan/RaTG13/2013,ncov,?,2013-07-24,Asia,China,Yunnan,Pu'Er,...,Yan Zhu et al,https://www.gisaid.org,?,?,2020-01-24,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSSTRGVYYPDKVFRSS...,"[p.F32S, p.S50L, p.T76I, p.Q218P, p.E324D, p.R...",30
1,Spike|hCoV-19/pangolin/Guangxi/P4L/2017|2017-0...,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,pangolin/Guangxi/P4L/2017,ncov,?,2017,Asia,China,Guangxi,NaN,...,Wu-Chun Cao et al,https://www.gisaid.org,?,?,2020-02-17,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,"[p.V6F, p.L7V, p.Q23G, p.L24I, p.A27G, p.F32S,...",94
2,Spike|hCoV-19/pangolin/Guangxi/P1E/2017|2017-0...,MFVFLFVLPLVSSQCVNLTTRTGIQPGYTNSSTRGVYYPDKVFRSS...,pangolin/Guangxi/P1E/2017,ncov,?,2017,Asia,China,Guangxi,NaN,...,Wu-Chun Cao et al,https://www.gisaid.org,?,?,2020-02-17,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVNLTTRTGIQPGYTNSSTRGVYYPDKVFRSS...,"[p.V6F, p.L7V, p.Q23G, p.L24I, p.P25Q, p.A27G,...",94
3,Spike|hCoV-19/pangolin/Guangxi/P5L/2017|2017-0...,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,pangolin/Guangxi/P5L/2017,ncov,?,2017,Asia,China,Guangxi,NaN,...,Wu-Chun Cao et al,https://www.gisaid.org,?,?,2020-02-17,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVNLTTRTGIPPGYTNSSTRGVYYPDKVFRSS...,"[p.V6F, p.L7V, p.Q23G, p.L24I, p.A27G, p.F32S,...",93
4,Spike|hCoV-19/pangolin/Guangxi/P3B/2017|2017-0...,MFVFLFVLPLVSSQCVXXXTRTGIPPGYTNSSTRGVYYPDKVFRSS...,pangolin/Guangxi/P3B/2017,ncov,?,2017,Asia,China,Guangxi,NaN,...,Wu-Chun Cao et al,https://www.gisaid.org,?,?,2020-02-17,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLFVLPLVSSQCVXXXTRTGIPPGYTNSSTRGVYYPDKVFRSS...,"[p.V6F, p.L7V, p.Q23G, p.L24I, p.A27G, p.F32S,...",45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514754,Spike|hCoV-19/USA/ME-HETL-J1279/2021|2021-02-0...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,USA/ME-HETL-J1279/2021,ncov,?,2021-02-05,North America,USA,Maine,NaN,...,Matluk et al,https://www.gisaid.org,?,?,2021-02-11,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.D614G, p.A846V]",2
514755,Spike|hCoV-19/USA/ME-HETL-J1309/2021|2021-02-0...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,USA/ME-HETL-J1309/2021,ncov,?,2021-02-05,North America,USA,Maine,NaN,...,Matluk et al,https://www.gisaid.org,?,?,2021-02-11,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.A520S, p.D614G]",2
514756,Spike|hCoV-19/USA/ME-HETL-J1280/2021|2021-02-0...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,USA/ME-HETL-J1280/2021,ncov,?,2021-02-05,North America,USA,Maine,NaN,...,Matluk et al,https://www.gisaid.org,?,?,2021-02-11,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.S255A, p.A263S, p.A264G, p.Y265W, p.Y266T, ...",8
514757,Spike|hCoV-19/Australia/QLD1545/2021|2021-02-0...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Australia/QLD1545/2021,ncov,?,2021-02-05,Oceania,Australia,Australia,NaN,...,Son Nguyen et al et al,https://www.gisaid.org,?,?,2021-02-09,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[del.69-70, del.144, p.N501Y, p.A570D, p.D614G...",9


In [8]:
usable_spike_df = annotated_df.loc[
    (annotated_df.num_muts < 30) &
    (annotated_df.unaligned_sequence.str.count("X") <= 2) &
    (annotated_df.unaligned_sequence.str.len() >= 1250)
]
usable_spike_df

,sequence_id,unaligned_sequence,strain,virus,genbank_accession,date,region,country,division,location,...,authors,url,title,paper_url,date_submitted,purpose_of_sequencing,aligned_ref,aligned_query,muts,num_muts
16,Spike|hCoV-19/Wuhan/IPBCAMS-WH-01/2019|2019-12...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/IPBCAMS-WH-01/2019,ncov,MT019529.1,2019-12-24,Asia,China,Hubei,Wuhan,...,Lili Ren et al,https://www.gisaid.org,?,?,2020-01-11,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
17,Spike|hCoV-19/Wuhan/Hu-1/2019|2019-12-31|EPI_I...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/Hu-1/2019,ncov,MN908947.3,2019-12-26,Asia,China,Hubei,Wuhan,...,Zhang et al,https://www.gisaid.org,A new coronavirus associated with human respir...,https://dx.doi.org/10.1038/s41586-020-2008-3,2020-01-12,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
18,Spike|hCoV-19/Wuhan/WH01/2019|2019-12-26|EPI_I...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/WH01/2019,ncov,LR757998.1,2019-12-26,Asia,China,Hubei,Wuhan,...,Weijun Chen et al,https://www.gisaid.org,Genomic characterisation and epidemiology of 2...,https://dx.doi.org/10.1016/S0140-6736(20)30251-8,2020-01-30,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
19,Spike|hCoV-19/Wuhan/IVDC-HB-GX02/2019|2019-12-...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/IVDC-HB-GX02/2019,ncov,?,2019-12-30,Asia,China,Hubei,Wuhan,...,Wenjie Tan et al,https://www.gisaid.org,?,?,2020-04-28,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
20,Spike|hCoV-19/Wuhan/IVDC-HB-05/2019|2019-12-30...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/IVDC-HB-05/2019,ncov,?,2019-12-30,Asia,China,Hubei,Wuhan,...,Wenjie Tan et al,https://www.gisaid.org,A Novel Coronavirus from Patients with Pneumon...,https://dx.doi.org/10.1056/NEJMoa2001017,2020-01-10,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514744,Spike|hCoV-19/USA/ME-HETL-J1296/2021|2021-02-0...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,USA/ME-HETL-J1296/2021,ncov,?,2021-02-04,North America,USA,Maine,NaN,...,Matluk et al,https://www.gisaid.org,?,?,2021-02-11,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.L118F, p.N501T, p.D614G]",3
514746,Spike|hCoV-19/USA/ME-HETL-J1295/2021|2021-02-0...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,USA/ME-HETL-J1295/2021,ncov,?,2021-02-04,North America,USA,Maine,NaN,...,Matluk et al,https://www.gisaid.org,?,?,2021-02-11,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.D614G, p.F643I, p.D1153Y]",3
514747,Spike|hCoV-19/Belgium/Jessa_55-2105-000762/202...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Belgium/Jessa_55-2105-000762/2021,ncov,?,2021-02-04,Europe,Belgium,Hasselt,Diepenbeek,...,Jessa_cmdLab et al,https://www.gisaid.org,?,?,2021-02-08,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.S98F, p.V143L, p.T208S, p.D614G, p.V1228L]",5
514750,Spike|hCoV-19/Belgium/Jessa_55-2105-001118/202...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Belgium/Jessa_55-2105-001118/2021,ncov,?,2021-02-04,Europe,Belgium,Hasselt,NaN,...,Jessa_cmdLab et al,https://www.gisaid.org,?,?,2021-02-09,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.M153I, p.A411T, p.L611R, p.A713V, p.K1269N]",5


In [9]:
import tqdm
import collections
mut_counts = collections.Counter()
for muts in usable_spike_df["muts"]:
    mut_counts.update(muts)
print("Unique muts", len(mut_counts))
mut_counts = pandas.Series(mut_counts).sort_values(ascending=False)

major_muts = mut_counts[mut_counts >= 10].index
print("Major muts", major_muts)

mut_df = pandas.DataFrame(index=usable_spike_df.index, columns=major_muts, dtype=bool)
for mut in tqdm.tqdm(major_muts):
    mut_df[mut] = usable_spike_df.muts.map(lambda muts: mut in muts)

usable_spike_df


Unique muts 3576
Major muts Index(['p.D614G', 'p.A222V', 'del.69-70', 'p.P681H', 'p.N501Y', 'del.144',
       'p.T716I', 'p.A570D', 'p.D1118H', 'p.S982A',
       ...
       'p.I472L', 'p.E340K', 'p.L179I', 'p.A829S', 'p.V120F', 'p.P330L',
       'p.D1163A', 'p.Q580H', 'p.V127A', 'p.D808E'],
      dtype='object', length=1135)


100%|██████████| 1135/1135 [01:28<00:00, 12.77it/s]


,sequence_id,unaligned_sequence,strain,virus,genbank_accession,date,region,country,division,location,...,authors,url,title,paper_url,date_submitted,purpose_of_sequencing,aligned_ref,aligned_query,muts,num_muts
16,Spike|hCoV-19/Wuhan/IPBCAMS-WH-01/2019|2019-12...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/IPBCAMS-WH-01/2019,ncov,MT019529.1,2019-12-24,Asia,China,Hubei,Wuhan,...,Lili Ren et al,https://www.gisaid.org,?,?,2020-01-11,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
17,Spike|hCoV-19/Wuhan/Hu-1/2019|2019-12-31|EPI_I...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/Hu-1/2019,ncov,MN908947.3,2019-12-26,Asia,China,Hubei,Wuhan,...,Zhang et al,https://www.gisaid.org,A new coronavirus associated with human respir...,https://dx.doi.org/10.1038/s41586-020-2008-3,2020-01-12,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
18,Spike|hCoV-19/Wuhan/WH01/2019|2019-12-26|EPI_I...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/WH01/2019,ncov,LR757998.1,2019-12-26,Asia,China,Hubei,Wuhan,...,Weijun Chen et al,https://www.gisaid.org,Genomic characterisation and epidemiology of 2...,https://dx.doi.org/10.1016/S0140-6736(20)30251-8,2020-01-30,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
19,Spike|hCoV-19/Wuhan/IVDC-HB-GX02/2019|2019-12-...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/IVDC-HB-GX02/2019,ncov,?,2019-12-30,Asia,China,Hubei,Wuhan,...,Wenjie Tan et al,https://www.gisaid.org,?,?,2020-04-28,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
20,Spike|hCoV-19/Wuhan/IVDC-HB-05/2019|2019-12-30...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Wuhan/IVDC-HB-05/2019,ncov,?,2019-12-30,Asia,China,Hubei,Wuhan,...,Wenjie Tan et al,https://www.gisaid.org,A Novel Coronavirus from Patients with Pneumon...,https://dx.doi.org/10.1056/NEJMoa2001017,2020-01-10,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514744,Spike|hCoV-19/USA/ME-HETL-J1296/2021|2021-02-0...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,USA/ME-HETL-J1296/2021,ncov,?,2021-02-04,North America,USA,Maine,NaN,...,Matluk et al,https://www.gisaid.org,?,?,2021-02-11,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.L118F, p.N501T, p.D614G]",3
514746,Spike|hCoV-19/USA/ME-HETL-J1295/2021|2021-02-0...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,USA/ME-HETL-J1295/2021,ncov,?,2021-02-04,North America,USA,Maine,NaN,...,Matluk et al,https://www.gisaid.org,?,?,2021-02-11,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.D614G, p.F643I, p.D1153Y]",3
514747,Spike|hCoV-19/Belgium/Jessa_55-2105-000762/202...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Belgium/Jessa_55-2105-000762/2021,ncov,?,2021-02-04,Europe,Belgium,Hasselt,Diepenbeek,...,Jessa_cmdLab et al,https://www.gisaid.org,?,?,2021-02-08,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.S98F, p.V143L, p.T208S, p.D614G, p.V1228L]",5
514750,Spike|hCoV-19/Belgium/Jessa_55-2105-001118/202...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,Belgium/Jessa_55-2105-001118/2021,ncov,?,2021-02-04,Europe,Belgium,Hasselt,NaN,...,Jessa_cmdLab et al,https://www.gisaid.org,?,?,2021-02-09,?,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,"[p.M153I, p.A411T, p.L611R, p.A713V, p.K1269N]",5


In [10]:
lineage_muts = []
for lineage, sub_df in usable_spike_df.groupby("pangolin_lineage"):
    muts_matrix = mut_df.loc[sub_df.index]
    means = muts_matrix.mean()
    selected_muts = muts_matrix.columns[means > 0.1].values
    for item in selected_muts:
        lineage_muts.append((lineage, len(sub_df), item, means[item]))

lineage_muts = pandas.DataFrame(lineage_muts, columns=["lineage", "sequences", "mut", "rate"])
lineage_muts


,lineage,sequences,mut,rate
0,A.10,3,p.D614G,1.000
1,A.11,5,p.L5F,0.200
2,A.11,5,p.V1122L,0.200
3,A.12,5,p.E1144Q,0.200
4,A.12,5,p.D614N,0.200
...,...,...,...,...
1589,R.2,8,p.Q677H,1.000
1590,R.2,8,p.E484K,1.000
1591,R.2,8,p.E1202Q,1.000
1592,R.2,8,p.T732S,1.000


In [11]:
lineage_muts_grouped = lineage_muts.groupby("lineage").mut.unique().to_frame()
lineage_muts_grouped["muts_80"] = lineage_muts.loc[lineage_muts.rate >= 0.8].groupby("lineage").mut.unique()
lineage_muts_grouped["muts_50"] = lineage_muts.loc[lineage_muts.rate >= 0.5].groupby("lineage").mut.unique()
lineage_muts_grouped["muts_10"] = lineage_muts_grouped.mut
del lineage_muts_grouped["mut"]

lineage_muts_grouped["num_sequences_for_lineage"] = lineage_muts_grouped.index.map(
    usable_spike_df.pangolin_lineage.value_counts())
lineage_muts_grouped = lineage_muts_grouped.sort_values("num_sequences_for_lineage", ascending=False)
lineage_muts_grouped

,muts_80,muts_50,muts_10,num_sequences_for_lineage
lineage,,,,
B.1.177,"[p.D614G, p.A222V]","[p.D614G, p.A222V, p.L18F]","[p.D614G, p.A222V, p.L18F]",65548
B.1.1.7,"[p.D614G, del.69-70, p.P681H, p.N501Y, del.144...","[p.D614G, del.69-70, p.P681H, p.N501Y, del.144...","[p.D614G, del.69-70, p.P681H, p.N501Y, del.144...",60070
B.1,[p.D614G],[p.D614G],[p.D614G],38315
B.1.2,[p.D614G],[p.D614G],[p.D614G],19785
B.1.1.29,[p.D614G],[p.D614G],[p.D614G],15525
...,...,...,...,...
B.1.464,[p.D614G],[p.D614G],[p.D614G],3
A.10,[p.D614G],[p.D614G],[p.D614G],3
B.1.476,"[p.D614G, p.S477N]","[p.D614G, p.S477N]","[p.D614G, p.S477N]",2


In [12]:
write_df = lineage_muts_grouped.loc[lineage_muts_grouped.num_sequences_for_lineage >= 50].copy()

def format(muts):
    if len(muts) == 0:
        return ""
    return " ".join(sorted(muts, key=lambda s: int(re.search("([0-9]+)", s).groups()[0])))

write_df["mutations >= 80%"] = write_df.muts_80.fillna("").map(format)
write_df["mutations >= 50%"] = write_df.muts_50.fillna("").map(format)
write_df["mutations >= 10%"] = write_df.muts_10.fillna("").map(format)


del write_df['muts_10']
del write_df['muts_50']
del write_df['muts_80']

write_df.to_csv("muts_by_lineage.csv", index=True)
write_df.to_excel("muts_by_lineage.xlsx", index=True)

write_df

,num_sequences_for_lineage,mutations >= 80%,mutations >= 50%,mutations >= 10%
lineage,,,,
B.1.177,65548,p.A222V p.D614G,p.L18F p.A222V p.D614G,p.L18F p.A222V p.D614G
B.1.1.7,60070,del.69-70 del.144 p.N501Y p.A570D p.D614G p.P6...,del.69-70 del.144 p.N501Y p.A570D p.D614G p.P6...,del.69-70 del.144 p.N501Y p.A570D p.D614G p.P6...
B.1,38315,p.D614G,p.D614G,p.D614G
B.1.2,19785,p.D614G,p.D614G,p.D614G
B.1.1.29,15525,p.D614G,p.D614G,p.D614G
...,...,...,...,...
B.1.1.123,50,p.D614G p.G769V,p.D614G p.G769V,p.D614G p.G769V
B.1.36.26,50,p.L54F p.D614G,p.L54F p.D614G,p.L54F p.D80Y p.F157L p.D614G
B.1.1.118,50,p.D614G,p.D614G p.S640F,p.D614G p.S640F
